# Rock-Paper-Scissors
Training a CNN on rock-paper-scissors hand images to build a simple game application

In [ ]:
# import modules
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import InputLayer, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

# define paths
train_folder = 'dataset/train'
test_folder = 'dataset/test'
validation_folder = 'dataset/validation'

## Load dataset

In [ ]:
# open test picture
def load_image(img_path: str) -> np.ndarray:
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image_rgb

def display_image(image: np.ndarray) -> None:
    plt.imshow(image)
    plt.axis('off')
    plt.show()

rock_folder = 'dataset/train/rock'
test_image = random.choice(os.listdir(rock_folder))
image_path = os.path.join(rock_folder, test_image)
img = load_image(image_path)
display_image(img)

## Preparing data for the model

In [ ]:
image_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

In [ ]:
random_img = image_gen.random_transform(img)
display_image(random_img)

In [ ]:
image_gen.flow_from_directory(train_folder)

In [ ]:
image_gen.flow_from_directory(test_folder)

## Create Model

In [ ]:
# Image shape Tensor
image_shape = (300, 300, 3)

# create sequential modle
model = Sequential()

model.add(InputLayer(input_shape=image_shape))

model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))

# Add 50% dropout to help reduce overfitting
model.add(Dropout(0.5))

# Output layer
model.add(Dense(units=3, activation='softmax'))  # 3 classes: 0=rock, 1=paper 2=scissors

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

## Moving the model to GPU
WIP, I currently cannot see my GPU listed here

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## Training Model

In [ ]:
batch_size = 32

train_image_gen = image_gen.flow_from_directory(
    directory=train_folder,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
test_image_gen = image_gen.flow_from_directory(
    directory=test_folder,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# indices
class_indices = train_image_gen.class_indices
class_labels = {v:k for k, v in class_indices.items()}
print(class_indices)
print(class_labels)

In [ ]:
results = model.fit(
    train_image_gen,
    epochs=100,
    steps_per_epoch=train_image_gen.samples/train_image_gen.batch_size,
    validation_data=test_image_gen,
    validation_steps=test_image_gen.samples/test_image_gen.batch_size,
    verbose=2)

## Model Evaluation

In [ ]:
plt.plot(results.history['accuracy'])

## Save model

In [ ]:
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M')
model.save(f'models/rps_v05_100epochs_{timestamp}.h5')

## Image prediction Test

In [ ]:
model = load_model('models/rps_v04_100epochs_20240209_2137.h5')
class_indices = {'paper': 0, 'rock': 1, 'scissors': 2}
class_labels = {v:k for k, v in class_indices.items()}

img_size = 300, 300
img_file = r'test\scissors.png'
img = image.load_img(img_file, target_size=img_size)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255

prediction_prob = model.predict(img, verbose=0)
print(prediction_prob)
predicted_class_index = np.argmax(prediction_prob)
print(predicted_class_index)
predicted_class_prob = prediction_prob[0][predicted_class_index]
print(predicted_class_prob)
predicted_class_label = class_labels[predicted_class_index]
print(predicted_class_label)

## Validation folder prediction

In [ ]:
success = 0
failure = 0
total = 0
failed_images = []

# load model
model = load_model('models/rps_v04_100epochs_20240209_2137.h5')
class_indices = {'paper': 0, 'rock': 1, 'scissors': 2}
class_labels = {v:k for k, v in class_indices.items()}

for subfolder in ['rock', 'paper', 'scissors']:

    current_folder = os.path.join(validation_folder, subfolder)

    for img_name in os.listdir(current_folder):

        img_file = os.path.normpath(os.path.join(current_folder, img_name))

        img = image.load_img(img_file, target_size=(150, 150))
        img_array = image.img_to_array(img)

        # add batch dimension
        img_normalized = img_array/255
        img_expanded = np.expand_dims(img_normalized, axis=0)

        # image prediction probability
        prediction_prob = model.predict(img_expanded, verbose=0)
        # print(prediction_prob)

        # Get predicted class index
        predicted_class_index = np.argmax(prediction_prob)

        # Map predicted class index to class label
        predicted_class_label = class_labels[predicted_class_index]

        # Print predicted class label
        if subfolder == predicted_class_label:
            success += 1
            result = 'success'
        else:
            failure += 1
            result = 'failure'
            failed_images.append((img_normalized, predicted_class_label, img_file))

        total += 1
        print(f'\rImages Classified: {total} | Success: {success} | Failure: {failure} | Accuracy: {success/total*100:.2f}%', end=' ', flush=True)

# print failed images with prediction and path
if failed_images:
    print(f"\n{50 * '-'}\nFAILED IMAGES:")
    for img, label, path in failed_images:
        print(path)
        img_size = 200
        plt.figure(figsize=(img_size/100, img_size/100))
        plt.imshow(img, cmap='gray')
        plt.title(label, loc='left')
        plt.axis('off')
        plt.show()


## Test on camera

In [ ]:
model = load_model('models/rps_v04_100epochs_20240209_2137.h5')
class_labels = {0: 'paper', 1: 'rock', 2: 'scissors'}

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cv2.namedWindow('Rock Paper Scissors')

img_size = 300, 300
x, y, w, h = 895, 78, 300, 300

while True:

    try:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        roi = frame[y:y+h, x:x+w]

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

        # copy and resize roi
        roi_copy = cv2.resize(roi, img_size)
        img = image.img_to_array(roi_copy)
        img = np.expand_dims(img, axis=0)
        img = img / 255.0
        prediction_prob = model.predict(img, verbose=0)
        predicted_class_index = np.argmax(prediction_prob)
        predicted_class_label = class_labels[predicted_class_index]
        predicted_class_prob = prediction_prob[0][predicted_class_index]

        frame[0:img_size[0], 0:img_size[1]] = roi_copy

        # Display prediction
        if predicted_class_prob < 0.9:
            label = 'None'
        else:
            label = f'{predicted_class_label} ({predicted_class_prob:.2f})'

        position = x, y + h + 20
        cv2.putText(frame, label, position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)

        # Display frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0,0,255), 2)
        cv2.imshow('Rock Paper Scissors', frame)

    except Exception as e:
        print(e)
        break

cap.release()
cv2.destroyAllWindows()